In [713]:
from functools import reduce

In [745]:
def hex2bin(hex_string):
    unfilled = bin(int(hex_string, 16))[2:]
    return ((hex_string[0] == '0') * 4 * '0'
            + (4 - len(unfilled) % 4) * (len(unfilled) % 4 != 0) * '0'
            + unfilled)

In [630]:
def process_literal(literal):
    if literal[0] == '0':
        return literal[1:5], literal[5:]
    
    bitstring, remnant = process_literal(literal[5:])
    return literal[1:5] + bitstring, remnant

In [751]:
def calc_version(iterable):
    ver = iterable[0]
    
    if not hasattr(iterable[2], "__iter__"):
        return iterable[0]
    
    return sum(calc_version(x) for x in iterable[2]) + ver


In [1050]:
def process_packet(packet):
    
    values = []
    
    if len(packet) < 6 or all(d == '0' for d in packet):
        return [], ""
    
    ver = packet[:3]
    type_id = packet[3:6]
    
    if type_id == '100':
        bitstr, remnant = process_literal(packet[6:])
        values += [[int(x, 2) for x in (ver, type_id, bitstr)]]

        return values, remnant

    length_type_id = packet[6]
    if length_type_id == '0':
        length = int(packet[7:22], 2)
        remnant = packet[22 + length:]

        _remnant = packet[22:22 + length]

        vals = []
        while _remnant != "":
            value, _remnant = process_packet(_remnant)
            vals += value
        values += [[int(ver, 2), int(type_id, 2), vals]]

        remnant = _remnant + remnant

    else:
        n_packets = int(packet[7:18], 2)
        remnant = packet[18:]
        vals = []
        for i in range(n_packets):
            value, remnant = process_packet(remnant)

            vals += value
        values += [[int(ver, 2), int(type_id, 2), vals]]
    
    return values, remnant

In [1016]:
with open("input", "r") as f:
    line = f.readline()

In [1055]:
msg = hex2bin(line)
v, r = process_packet(msg)

### Part 1

In [ ]:
calc_version(v[0])

### Part 2

In [955]:
operator = {
    0: lambda x, y=0: x + y,
    1: lambda x, y=1: x * y,
    2: min,
    3: max,
    4: lambda x: x,
    5: lambda x, y: int(x > y),
    6: lambda x, y: int(x < y),
    7: lambda x, y: int(x == y),
}

In [956]:
def do_operation(iterable):
    if not hasattr(iterable[2], "__iter__"):
        return iterable[2]
    
    if iterable[1] in [0, 1]:
        return reduce(operator[iterable[1]], map(do_operation, iterable[2]))
    
    if iterable[1] in [5, 6, 7]:
        return operator[iterable[1]](*[do_operation(x) for x in iterable[2]])
    
    return operator[iterable[1]]([do_operation(x) for x in iterable[2]])

In [1058]:
do_operation(v[0])

1549026292886